<center>
<h2>DV2599 - PROJECT</h2>
<h3>Predicting Payment date</h3>
<h4>
Rasmus Dunder - radn21@student.bth.se
</h4>
<h4>
Rasmus Johansson - rajo22@student.bth.se
</h4>
</center>

#### Import modules / libraries

In [233]:
import pandas as pd
import datetime
import math

#### Data cleaning

In [238]:
df = pd.read_csv('dataset.csv')

df = df.drop(columns=['name_customer', 'buisness_year', 'invoice_currency', 'doc_id','posting_id', 'area_business','document type', 'baseline_create_date', 'document_create_date.1', 'document_create_date'])

# df['invoice_id'] = df['invoice_id'].astype(str)
not_yetpayed = df['isOpen'].value_counts().sort_index()
quality_ratios = round((not_yetpayed / not_yetpayed.sum()),4) * 100
print(f"Ratios: {quality_ratios}")
print(f"Counts: {not_yetpayed}")

# Drop all Columns where invoice is not yet paid
df = df[df['isOpen'] == 0]

print(f"\nNull values in the dataset: {sum(df.isnull().sum(axis=1))}\n")

df.dropna(inplace=True)

# print(df['invoice_id'].value_counts())
df.drop_duplicates(subset=['invoice_id'], inplace=True)

df = df.drop(columns=['invoice_id', 'isOpen'])

df['due_in_date'] = pd.to_datetime(df['due_in_date'], format='%Y%m%d')
df['clear_date'] = pd.to_datetime(df['clear_date'],  errors='coerce')
df['posting_date'] = pd.to_datetime(df['posting_date'], format='%m/%d/%Y')

# Drop Columns where due_in_date is before posting date
df = df[df['due_in_date'] >= df['posting_date']]


df['payment_term'] = (df['due_in_date'] - df['posting_date']).dt.days
df['delayed_payment_days'] = (df['clear_date'] - df['due_in_date']).dt.days
df['payed_on_time'] = df['delayed_payment_days'] <= 0

print(df.iloc[0:2])

Ratios: isOpen
0    80.0
1    20.0
Name: count, dtype: float64
Counts: isOpen
0    40000
1    10000
Name: count, dtype: int64

Null values in the dataset: 6

  business_code cust_number clear_date posting_date due_in_date  \
0          U001   200769623 2020-02-11   2020-01-26  2020-02-10   
1          U001   200980828 2019-08-08   2019-07-22  2019-08-11   

   total_open_amount cust_payment_terms  payment_term  delayed_payment_days  \
0           54273.28               NAH4            15                     1   
1           79656.60               NAD1            20                    -3   

   payed_on_time  
0          False  
1           True  


Att tänka på

- Outliers
- Normalisering